In [1]:
!pip install datasets tensorflow -q

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

In [1]:
from datasets import load_dataset
dataset = load_dataset("bookcorpus", split="train", streaming=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/18.5k [00:00<?, ?B/s]

bookcorpus.py:   0%|          | 0.00/3.25k [00:00<?, ?B/s]

In [17]:
texts=[]
count=0
for example in dataset:
  if count>=10000:
    break
  # filter
  text=example['text'].strip()
  if 20<=len(text)<=200:
    texts.append(text)
    count+=1
  if count%1000==0:
    print(f"{count}/10000")

len(texts)

print("sample")
for i in range(5):
  print(f"{i+1}.{texts[i]}")

1000/10000
2000/10000
3000/10000
4000/10000
5000/10000
6000/10000
7000/10000
8000/10000
9000/10000
9000/10000
9000/10000
9000/10000
10000/10000
sample
1.usually , he would be tearing around the living room , playing with his toys .
2.but just one look at a minion sent him practically catatonic .
3.that had been megan 's plan when she got him dressed earlier .
4.he 'd seen the movie almost by mistake , considering he was a little young for the pg cartoon , but with older cousins , along with her brothers , mason was often exposed to things that were older .
5.she liked to think being surrounded by adults and older kids was one reason why he was a such a good talker for his age .


In [3]:
import re
# texts cleaning
def clean_text(text):
  text=text.lower()
  text=''.join(text.split())
  text=re.sub(r'[^a-zA-Z0-9\s.,!?;:\'-]','',text)
  return text

cleaned_texts=[clean_text(text) for text in texts]

len(cleaned_texts)

10000

In [6]:
# Инициализируем токенизатор
tokenizer = Tokenizer()

# Обучаем токенизатор на заголовках
tokenizer.fit_on_texts(texts)

# Преобразуем заголовки в последовательности чисел
sequences = tokenizer.texts_to_sequences(texts)

# Создаем входные и выходные данные
X = []
y = []
for seq in sequences:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[i])

In [7]:
# Создаем входные и выходные данные
X = []
y = []
for seq in sequences:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[i])

In [8]:
X[:10], y[:10]

([[639],
  [639, 9],
  [639, 9, 57],
  [639, 9, 57, 36],
  [639, 9, 57, 36, 3012],
  [639, 9, 57, 36, 3012, 69],
  [639, 9, 57, 36, 3012, 69, 1],
  [639, 9, 57, 36, 3012, 69, 1, 429],
  [639, 9, 57, 36, 3012, 69, 1, 429, 100],
  [639, 9, 57, 36, 3012, 69, 1, 429, 100, 640]],
 [9, 57, 36, 3012, 69, 1, 429, 100, 640, 20])

In [9]:
# Преобразуем списки в массивы numpy
X = np.asarray(X, dtype="object")
y = np.array(y)

# Дополняем последовательности до одинаковой длины
X = pad_sequences(X)

# Преобразуем y в one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

In [10]:
# Создаем модель
model = Sequential()

# Добавляем слой Embedding
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X.shape[1]))

# Добавляем слой LSTM
model.add(LSTM(150, return_sequences=False))

# Добавляем полносвязный слой
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

# Компилируем модель
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Выводим информацию о модели
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Обучаем модель
history = model.fit(X, y, epochs=40, batch_size=64, validation_split=0.2)

Epoch 1/40
1457/1457 ━━━━━━━━━━━━━━━━━━━━ 236s 159ms/step - accuracy: 0.0398 - loss: 6.5206 - val_accuracy: 0.0904 - val_loss: 6.1720
Epoch 2/40
1457/1457 ━━━━━━━━━━━━━━━━━━━━ 232s 159ms/step - accuracy: 0.1229 - loss: 5.3598 - val_accuracy: 0.1211 - val_loss: 5.9874
Epoch 3/40
1457/1457 ━━━━━━━━━━━━━━━━━━━━ 258s 156ms/step - accuracy: 0.1598 - loss: 4.8537 - val_accuracy: 0.1331 - val_loss: 5.9315
Epoch 4/40
1457/1457 ━━━━━━━━━━━━━━━━━━━━ 267s 160ms/step - accuracy: 0.1822 - loss: 4.5652 - val_accuracy: 0.1368 - val_loss: 5.9867
Epoch 5/40
1457/1457 ━━━━━━━━━━━━━━━━━━━━ 278s 171ms/step - accuracy: 0.1977 - loss: 4.3048 - val_accuracy: 0.1402 - val_loss: 6.0480
Epoch 6/40
1457/1457 ━━━━━━━━━━━━━━━━━━━━ 251s 163ms/step - accuracy: 0.2168 - loss: 4.0818 - val_accuracy: 0.1398 - val_loss: 6.1383
Epoch 7/40
1457/1457 ━━━━━━━━━━━━━━━━━━━━ 252s 156ms/step - accuracy: 0.2286 - loss: 3.8988 - val_accuracy: 0.1402 - val_loss: 6.2669
Epoch 8/40
1457/1457 ━━━━━━━━━━━━━━━━━━━━ 264s 158ms/step - ac

In [25]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 43, 100)        │       754,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 150)            │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7546)           │     1,139,446 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,133,940 (23.40 MB)

 Trainable params: 2,044,646 (7.80 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,089,294 (15.60 MB)

In [16]:
# Функция для генерации текста
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Генерируем новый заголовок
generated_text = generate_text("the reason", 20, X.shape[1])
print(generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
the reason is so cliche '' he murmured into a smile before heading onto the door and took her hand to her


In [29]:
generated_text = generate_text("they", 20, X.shape[1])
print(generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
they were interrupted by an air traffic controller with a headset of his c k in his chest with all years


In [26]:
# Сохраняем модель
model.save('bookcorpus_generator.keras')

The model traing part literally took decades... But the results seem like not so bad, we can say that by the generated texts.